In [1]:
import pandas as pd

In [122]:
scs = pd.read_csv('../data/SCS alerts Nov.csv')

In [123]:
scs.head()

,Number,Alert,Entry time,PLC,Desk,Duration,Fault ID
0,141254,C17: C1701STA254 254B12 occupancy sensor fault,01/11/2020 00:00,C17,P01,0.00:00:13,3416
1,138183,C17: C1701STA012 012B13 occupancy sensor fault,01/11/2020 00:00,C17,P01,0.00:00:03,3416
2,122518,C15: C1502STA034 034B12 occupancy sensor fault,01/11/2020 00:00,C15,P02,0.00:00:05,3416
3,444609,SCS M09: single module 1 emergency stop unit ...,01/11/2020 00:00,SCSM09,SCSM09,0.00:00:13,10408
4,139815,C17: C1701STA112 112B13 occupancy sensor fault,01/11/2020 00:00,C17,P01,0.00:00:08,3416


In [127]:
scs['code'] = scs['Alert'].str.extract('([A-Z]{3}[0-9]{3}|[A-Z]{3} [A-Z][0-9]{2})')

In [130]:
bad_matches = scs[scs['code'].isna() & (scs['PLC'] != scs['Desk'])]['Alert']

In [133]:
scs.iloc[bad_matches[bad_matches.str.extract('([0-9]{2})')[0].astype('int') < 35].index]

,Number,Alert,Entry time,PLC,Desk,Duration,Fault ID,code
450,133009,C16: C16 emergency stop monitoring area 16,01/11/2020 01:09,C16,Z,0.00:00:21,3211,NaN
865,127938,C15: C15+P21-SN12 emergency stop button actuat...,01/11/2020 03:05,C15,Z,0.00:04:57,3436,NaN
866,126416,C15: C15 emergency stop monitoring area 15,01/11/2020 03:05,C15,Z,0.00:05:04,3211,NaN
4768,141303,C17: C17 emergency stop monitoring area 22,01/11/2020 20:13,C17,Z,0.00:02:40,3211,NaN
7409,142578,C17: C17+P22-SN12 emergency stop button actuat...,02/11/2020 19:21,C17,Z,0.00:00:23,3436,NaN
...,...,...,...,...,...,...,...,...
170215,141302,C17: C17 emergency stop monitoring area 21,25/11/2020 02:05,C17,Z,0.00:00:49,3211,NaN
170216,141302,C17: C17 emergency stop monitoring area 21,25/11/2020 02:22,C17,Z,0.00:01:36,3211,NaN
170217,141302,C17: C17 emergency stop monitoring area 21,25/11/2020 04:22,C17,Z,0.00:01:29,3211,NaN
170218,141302,C17: C17 emergency stop monitoring area 21,25/11/2020 11:12,C17,Z,0.00:00:32,3211,NaN


In [137]:
scs.loc[scs['code'].isna(), 'code'] = scs.loc[scs['code'].isna(), 'PLC']

In [138]:
scs.to_csv('../data/SCS alerts Nov_with_asset_code.csv')